# Initialization

In [ ]:
$TenantId = "<TENANT_ID>"

# connect once for all necessary scopes for this notebook
# Disconnect-Graph
# TODO: use Get-MgContext to check if we need to connect again
$Scopes = "Directory.AccessAsUser.All", "Policy.Read.All", "RoleManagement.ReadWrite.Directory", "RoleManagementAlert.Read.Directory", "AccessReview.Read.All"
$null = Connect-MgGraph -Scopes $Scopes -TenantId $TenantId -ErrorAction Stop

Write-Host "Connected to tenant '$TenantId' with the following scope: $Scopes"

# load functions
. "..\src\functions.ps1"

##  User consent for apps

*Severity*: Medium

*Guid*: 459c373e-7ed7-4162-9b37-5a917ecbe48f

Consent & Permissions: Allow user consent for apps from verified publishers

[Configure how users consent to applications](https://learn.microsoft.com/en-us/azure/active-directory/manage-apps/configure-user-consent?pivots=ms-graph)

# Privileged administration

## Limit the number of Global Administrators to less than 5

*Severity*: High

*Guid*: 9e6efe9d-f28f-463b-9bff-b5080173e9fe

In [ ]:
$Setting = Get-EntraIdRoleAssignment -RoleName "Global Administrator"
$Compliant = $Setting.Count -lt 5

if($Compliant)
{
    Write-Host "Compliant to control; there are $($Setting.Count) Global Administrators (Assigned and Eligeble)" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; there are $($Setting.Count) Global Administrators (Assigned and Eligeble)" -ForegroundColor Red
}

Write-Host "`nGlobal Administrators:`n"
$Setting | Select-Object -Property Id, @{ Name = 'displayName'; Expression = { $_.AdditionalProperties.displayName } }, @{ Name = 'userPrincipalName'; Expression = { $_.AdditionalProperties.userPrincipalName } } | Format-Table -AutoSize


## Use groups for Azure AD role assignments (WiP)

*Severity*: High

*Guid*: e0d968d3-87f6-41fb-a4f9-d852f1673f4c

[Best Practice: Use groups for Azure AD role assignments and delegate the role assignment](https://learn.microsoft.com/en-us/azure/active-directory/roles/best-practices#6-use-groups-for-azure-ad-role-assignments-and-delegate-the-role-assignment)


In [ ]:
$RoleName = "Global Administrator"

# Get the directory role id for $RoleName
$DirectoryRoleId = Get-MgDirectoryRole -Filter "DisplayName eq '$RoleName'" | Select-Object -ExpandProperty Id
# Get currently assigned
$Assigned = Get-MgDirectoryRoleMember -DirectoryRoleId $DirectoryRoleId | Select-Object -ExpandProperty Id

# TODO: $Assigned includes eligeble that have activated the role, but does not provide any details. we need to kow the 'state' and if it is activated we can disregard

# Get the role definition id for $RoleName
$DirectoryRoleDefinitionId = Get-MgBetaRoleManagementDirectoryRoleDefinition -Filter "DisplayName eq '$RoleName'" -Property "id" | Select-Object -ExpandProperty Id
# get principals that are eligble for GA
$EligeblePrincipals = Get-MgBetaRoleManagementDirectoryRoleEligibilityScheduleInstance -Filter "roleDefinitionId eq '$DirectoryRoleDefinitionId'" | Select-Object -ExpandProperty PrincipalId

$DirectoryObjectByIds = $EligeblePrincipals # + $Assigned

$params = @{
    ids   = $DirectoryObjectByIds
    types = @(
        "user"
        "group"
    )
}

if($params.ids.Count -gt 0)
{
    $DirectoryObject = Get-MgDirectoryObjectById -BodyParameter $params

    $DirectoryObject | Select-Object Id, @{ Name = 'displayName'; Expression = { $_.AdditionalProperties.displayName } }, @{ Name = 'type'; Expression = { $_.AdditionalProperties.'@odata.type'.split('.') | Select-Object -Last 1 } }
}

## PIM Alerts

*Severity*: High

*Guid*: N/A

There should be no active alerts in PIM. If below identifies any active alerts go to [PIM alerts](https://portal.azure.com/#view/Microsoft_Azure_PIMCommon/ResourceMenuBlade/~/Alerts/resourceId//resourceType/tenant/provider/aadroles) for further details.

In [ ]:
$GovernanceRoleManagementAlerts = Get-MgBetaIdentityGovernanceRoleManagementAlert -Filter "scopeId eq '/' and scopeType eq 'DirectoryRole' and isActive eq true" -ExpandProperty "alertDefinition,alertConfiguration,alertIncidents"

$GovernanceRoleManagementAlerts | Select-Object -Property @{ Name = 'Alert'; Expression = { $_.alertDefinition.displayName } }, IncidentCount

We can also list affected principals. Note that in some cases there is no direct principal, ex. for the alert `NoMfaOnRoleActivationAlert`

In [ ]:
$GovernanceRoleManagementAlerts = Get-MgBetaIdentityGovernanceRoleManagementAlert -Filter "scopeId eq '/' and scopeType eq 'DirectoryRole' and isActive eq true" -ExpandProperty "alertDefinition,alertConfiguration,alertIncidents"

$GovernanceRoleManagementAlerts.alertIncidents.AdditionalProperties | Where-Object { $_.assigneeUserPrincipalName } | ForEach-Object {
    $_ | Select-Object -Property @{ Name = 'Role'; Expression = { $_.roleDisplayName } }, @{ Name = 'User'; Expression = { "$($_.assigneeDisplayName) ($($_.assigneeUserPrincipalName))" } }
}

## Recurring access reviews

*Severity*: High

*Guid*: eae64d01-0d3a-4ae1-a89d-cc1c2ad3888f

Configure recurring access reviews to revoke unneeded permissions over time.

[Best Practice: Configure recurring access reviews to revoke unneeded permissions over time](https://learn.microsoft.com/en-us/azure/active-directory/roles/best-practices#4-configure-recurring-access-reviews-to-revoke-unneeded-permissions-over-time)

If there are no access review definitions then there are no recurring access reviews.

In [ ]:
$AccessReviewDefinitions = Get-MgBetaIdentityGovernanceAccessReviewDefinition

Write-Host "Access review definitions: $(($AccessReviewDefinitions | Measure-Object).Count)"

## Access Reviews: Enabled for all groups

*Severity*: Medium

*Guid*: e6b4bed3-d5f3-4547-a134-7dc56028a71f

[Plan a Microsoft Entra access reviews deployment](https://learn.microsoft.com/en-us/azure/active-directory/governance/deploy-access-reviews)

# External Identities

## Guest invite settings

*Severity*: High

*Guid*: be64dd7d-f2e8-4bbb-a468-155abc9164e9

External Collaboration Settings: Guest invite settings set to `'Only users assigned to specific admin roles can invite guest users'` or `'No one in the organization can invite guest users including admins (most restrictive)'`

In [ ]:
$AuthorizationPolicy = Get-MgPolicyAuthorizationPolicy

$Setting = $AuthorizationPolicy.AllowInvitesFrom
$Compliant = $Setting -in 'adminsAndGuestInviters', 'none'

if($Compliant)
{
    Write-Host "Compliant to control, setting is $($Setting)" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control, setting is $($Setting)" -ForegroundColor Red
}

## Guest user access restrictions

*Severity*: High

*Guid*: 459c373e-7ed7-4162-9b37-5a917ecbe48f

External Collaboration Settings: Guest user access set to `'Guest user access is restricted to properties and memberships of their own directory objects (most restrictive)'`

In [ ]:
# TODO: does not say anything about guest user access....

$ExternalIdentityPolicy = Get-MgBetaPolicyExternalIdentityPolicy #-ExpandProperty "AdditionalProperties"

# $ExternalIdentityPolicy | fl *
# $ExternalIdentityPolicy.AdditionalProperties | fl *



# User Setting

## User role permissions (Application registration)

*Severity*: High

*Guid*: a2cf2149-d013-4a92-9ce5-74dccbd8ac2a

Users can register applications should be set to `No`.

Users should not be allowed to register applications. Use specific roles such as `Application Developer`.

In [ ]:
$AuthorizationPolicy = Get-MgPolicyAuthorizationPolicy -Property "DefaultUserRolePermissions"

$Setting = $AuthorizationPolicy.DefaultUserRolePermissions.AllowedToCreateApps
$Compliant = $Setting -eq $false

if($Compliant)
{
    Write-Host "Compliant to control; users are not allowed to create applications" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; users are allowed to create applications" -ForegroundColor Red
}

# Custom Domains

## Verified Domains

*Severity*: High

*Guid*: bade4aad-1e8c-439e-a946-667313c00567

Only validated customer domains are registered

In [ ]:
$Domains = Get-MgBetaDomain

$UnverifiedDomains = $Domains | Where-Object {-not $_.IsVerified}

$Setting = $UnverifiedDomains
$Compliant = $Setting.Count -eq 0

if($Compliant)
{
    Write-Host "Compliant to control; All domains are verified" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; There are unverified domains registered: $($Setting | Select-Object -ExpandProperty Id)" -ForegroundColor Red
}

# Enterprise Applications

##  User consent for apps

*Severity*: Medium

*Guid*: 459c373e-7ed7-4162-9b37-5a917ecbe48f

Consent & Permissions: Allow user consent for apps from verified publishers

[Configure how users consent to applications](https://learn.microsoft.com/en-us/azure/active-directory/manage-apps/configure-user-consent?pivots=ms-graph)

In [ ]:
$PolicyAuthorization = Get-MgPolicyAuthorizationPolicy #-ExpandProperty defaultUserRolePermissions
$permissionGrantPoliciesAssigned = $PolicyAuthorization.DefaultUserRolePermissions.permissionGrantPoliciesAssigned

$Setting = $permissionGrantPoliciesAssigned
$Compliant = $Setting[0] -ne "ManagePermissionGrantsForSelf.microsoft-user-default-legacy"

if($Compliant)
{
    Write-Host "Compliant to control; users are only allowed to consent to apps from verified publishers or not consent at all." -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; users are allowed to consent to all applications." -ForegroundColor Red
}

##  group owner consent

*Severity*: Medium

*Guid*: 909aed8c-44cf-43b2-a381-8bafa2cf2149

Consent & Permissions: Allow group owner consent for selected group owners 

[Configure group owner consent to applications](https://learn.microsoft.com/en-us/azure/active-directory/manage-apps/configure-user-consent-groups?tabs=azure-portal)

In [ ]:
# TODO - example is using AzureADPreview module and we would like to stick to MS Graph

# Conditional Access Policies

## Block Legacy Protocols

*Severity*: High

*Guid*: 9e6efe9d-f28f-463b-9bff-b5080173e9fe

[Common Conditional Access policy: Block legacy authentication](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/howto-conditional-access-policy-block-legacy)

Below looks for a conditional access policy that blocks legacy protocols and also outputs users excluded.

In [ ]:
# we are looking for a policy that is enabled, the control is block, includes all users, and condition is legacy clients
$Filter = "state eq 'enabled' and grantControls/builtInControls/all(i:i eq 'block') and conditions/users/includeUsers/all(i:i eq 'All') and conditions/clientAppTypes/all(i:i eq 'exchangeActiveSync' or i eq 'other')"
# need to use the beta API as v1.0 does not include policies made from templates
$BlockLegacyProtocolPolicy = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter

# $BlockLegacyProtocolPolicy | Select-Object -Property DisplayName, Id

$ExcludeUsers = $BlockLegacyProtocolPolicy.Conditions.Users.ExcludeUsers
$ExcludeGroups = $BlockLegacyProtocolPolicy.Conditions.Users.ExcludeGroups
$ExcludeGuestsOrExternalUsers = $BlockLegacyProtocolPolicy.Conditions.Users.ExcludeGuestsOrExternalUsers

# TODO:
# $ExcludeGroups
# $ExcludeGuestsOrExternalUsers

$Compliant = $null -ne $BlockLegacyProtocolPolicy -and $BlockLegacyProtocolPolicy.Count -gt 0

if($Compliant)
{
    Write-Host "Compliant to control; CA Policy found blocking legacy protocols" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; No valid CA Policy found blocking legacy protocols" -ForegroundColor Red
}

$ExcludeUsers | Where-Object { $null -ne $_ } | ForEach-Object {
    $ExcludedUser = Get-MgUser -Filter "id eq '$_'"
    Write-Host "Excluded user: $($ExcludedUser.DisplayName) ($($ExcludedUser.UserPrincipalName))"
}

## Require MFA for Administrators

*Severity*: High

*Guid*: fe1bd15d-d2f0-4d5e-972d-41e3611cc57b

[Common Conditional Access policy: Require MFA for administrators](https://learn.microsoft.com/en-us/azure/active-directory/conditional-access/howto-conditional-access-policy-admin-mfa)

Below looks for a conditional access policy that matches the policy template `"Require multifactor authentication for admins"`

In [ ]:
# we are looking for a policy that is enabled, the control is block, includes all users, and condition is legacy clients
$PrivilegedRolesList = "('62e90394-69f5-4237-9190-012177145e10','194ae4cb-b126-40b2-bd5b-6091b380977d','f28a1f50-f6e7-4571-818b-6a12f2af6b6c','29232cdf-9323-42fd-ade2-1d097af3e4de','b1be1c3e-b65d-4f19-8427-f6fa0d97feb9','729827e3-9c14-49f7-bb1b-9608f156bbb8','b0f54661-2d74-4c50-afa3-1ec803f12efe','fe930be7-5e62-47db-91af-98c3a49a38b1','c4e39bd9-1100-46d3-8c65-fb160da0071f','9b895d92-2cd3-44c7-9d02-a6ac2d5ea5c3','158c047a-c907-4556-b7ef-446551a6b5f7','966707d0-3269-4727-9be2-8c3a10f19b9d','7be44c8a-adaf-4e2a-84d6-ab2649e08a13','e8611ab8-c189-46e8-94e1-60213ab1f814')"
# cannot filter on authenticationStrength: Invalid $filter: navigation property 'authenticationStrength' not found on type 'microsoft.graph.conditionalAccessPolicy'.
# we will do this when checking if the policy is compliant
$Filter = "state eq 'enabled' and conditions/applications/includeApplications/all(i:i eq 'All') and conditions/users/includeRoles/all(i:i in $PrivilegedRolesList)" # and (grantControls/builtInControls/all(i:i eq 'mfa') or grantControls/authenticationStrength ne null)
# need to use the beta API as v1.0 does not include policies made from templates
$RequireMfaAdminsPolicy = Get-MgBetaIdentityConditionalAccessPolicy -Filter $Filter

$RequireMfaAdminsPolicy | Select-Object -Property DisplayName, Id

$ExcludeUsers = $RequireMfaAdminsPolicy.Conditions.Users.ExcludeUsers
$ExcludeGroups = $RequireMfaAdminsPolicy.Conditions.Users.ExcludeGroups
$ExcludeGuestsOrExternalUsers = $RequireMfaAdminsPolicy.Conditions.Users.ExcludeGuestsOrExternalUsers

# TODO:
# $ExcludeGroups
# $ExcludeGuestsOrExternalUsers

if($RequireMfaAdminsPolicy.Count -gt 1)
{
    # $Compliant will become $false as we expect a single policy
    Write-Warning "Found multiple matching CA policies"
}
$Compliant = $null -ne $RequireMfaAdminsPolicy -and $RequireMfaAdminsPolicy.Count -eq 1 -and ('mfa' -in $RequireMfaAdminsPolicy.GrantControls.builtInControls -or $RequireMfaAdminsPolicy.GrantControls.authenticationStrength.requirementsSatisfied -eq 'mfa')

if($Compliant)
{
    Write-Host "Compliant to control; CA Policy found that requires administrators to use MFA or better" -ForegroundColor Green
}
else {
    Write-Host "Not compliant to control; No valid CA Policy found targeting Azure administrators" -ForegroundColor Red
}

$ExcludeUsers | Where-Object { $null -ne $_ } | ForEach-Object {
    $ExcludedUser = Get-MgUser -Filter "id eq '$_'"
    Write-Host "Excluded user: $($ExcludedUser.DisplayName) ($($ExcludedUser.UserPrincipalName))"
}